<a href="https://colab.research.google.com/github/Michele-Simeone/AILabProj-NBA-Scheme-Classification/blob/main/Code/Image_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Importazione delle librerie**

In [ ]:
import pandas as pd
import numpy as np
from PIL import Image

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Arc

import json
import random

from google.colab import drive
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# **Operazioni per generare gli eventi di una partita sottoforma di immagini**

## **Convertire I Dati Json in un Dataset**

In [ ]:
def open_file(s,role):
  with open(s,'r') as f:
      data = json.loads(f.read())
  headers = ["team_id", "player_id", "x_loc", "y_loc", 
            "radius", "moment", "game_clock", "shot_clock","event_id"]
  df = pd.json_normalize(data, record_path =['events','moments'], meta=[["events",'eventId']])
  moments = df.values.tolist()
  player_moments = []
  i=0
  for moment in moments:
      # Formattazione dei momenti con aggiunta di informazioni temporali
      for player in moment[5]:
          player.append(i)
          player.append(moment[2])
          player.append(moment[3])
          player.append(moment[6])
          player_moments.append(player)
      i=i+1
  df = pd.DataFrame(player_moments, columns=headers)
  df["event_id"] = pd.to_numeric(df["event_id"])   
  result = pd.merge(df,role,on='player_id')
  result.sort_values(by=["moment"], inplace = True)
  result=result[["event_id","moment","player_id","name","surname","x_loc","y_loc","radius","game_clock","shot_clock","team","pg%","sg%","sf%","pf%","c%"]]
  return result

## **Assegnazione ruoli per evento**

In [ ]:
def assign_role(a):
  b=pd.DataFrame(columns=["event_id","moment","player_id","name","surname","x_loc","y_loc","radius","game_clock","shot_clock","team","pg%","sg%","sf%","pf%","c%"])
  c=pd.DataFrame(columns=["player_id","role"])
  c=c.append({'player_id': -1,"role": "ball"},ignore_index=True)
  for i in range(1,6):
    b=b.append(a.iloc[i])
  b=b[["player_id","pg%","sg%","sf%","pf%","c%"]]
  pg=0
  x=0
  sg=0
  y=0
  sf=0
  pf=0
  m=0
  ce=0
  n=0
  for i in b.index: 
    if b["c%"][i]>=n:
      ce=b["player_id"][i]
      n=b["c%"][i]
  for i in b.index: 
    if b["pg%"][i]>=x and ce!=b["player_id"][i]:
      pg=b["player_id"][i]
      x=b["pg%"][i]
  for i in b.index: 
    if b["pf%"][i]>=m and ce!=b["player_id"][i] and pg!=b["player_id"][i]:
      pf=b["player_id"][i]
      m=b["pf%"][i]
  for i in b.index: 
    if b["sg%"][i]>=y and ce!=b["player_id"][i] and pg!=b["player_id"][i] and pf!=b["player_id"][i]:
      sg=b["player_id"][i]
      y=b["sg%"][i]
  for i in b.index: 
    if ce!=b["player_id"][i] and pg!=b["player_id"][i] and pf!=b["player_id"][i] and sg!=b["player_id"][i]:
      sf=b["player_id"][i]
      break
  c=c.append({'player_id':pg,"role": "pg"},ignore_index=True)
  c=c.append({'player_id':sg,"role": "sg"},ignore_index=True)  
  c=c.append({'player_id':sf,"role": "sf"},ignore_index=True)
  c=c.append({'player_id':pf,"role": "pf"},ignore_index=True)
  c=c.append({'player_id':ce,"role": "c"},ignore_index=True)
  return c

## **Plot campo**

In [ ]:
def draw_court(ax=None, color="gray", lw=1, zorder=0):
    
    if ax is None:
        ax = plt.gca()


    outer = Rectangle((0,-50), width=94, height=50, color=color,
                      zorder=zorder, fill=False, lw=lw)

   
    l_hoop = Circle((5.35,-25), radius=.75, lw=lw, fill=False, 
                    color=color, zorder=zorder)
    r_hoop = Circle((88.65,-25), radius=.75, lw=lw, fill=False,
                    color=color, zorder=zorder)
    
    l_backboard = Rectangle((4,-28), 0, 6, lw=lw, color=color,
                            zorder=zorder)
    r_backboard = Rectangle((90, -28), 0, 6, lw=lw,color=color,
                            zorder=zorder)

   
    l_outer_box = Rectangle((0, -33), 19, 16, lw=lw, fill=False,
                            color=color, zorder=zorder)    
    l_inner_box = Rectangle((0, -31), 19, 12, lw=lw, fill=False,
                            color=color, zorder=zorder)
    r_outer_box = Rectangle((75, -33), 19, 16, lw=lw, fill=False,
                            color=color, zorder=zorder)

    r_inner_box = Rectangle((75, -31), 19, 12, lw=lw, fill=False,
                            color=color, zorder=zorder)

    
    l_free_throw = Circle((19,-25), radius=6, lw=lw, fill=False,
                          color=color, zorder=zorder)
    r_free_throw = Circle((75, -25), radius=6, lw=lw, fill=False,
                          color=color, zorder=zorder)

   
    l_corner_a = Rectangle((0,-3), 14, 0, lw=lw, color=color,
                           zorder=zorder)
    l_corner_b = Rectangle((0,-47), 14, 0, lw=lw, color=color,
                           zorder=zorder)
    r_corner_a = Rectangle((80, -3), 14, 0, lw=lw, color=color,
                           zorder=zorder)
    r_corner_b = Rectangle((80, -47), 14, 0, lw=lw, color=color,
                           zorder=zorder)
    
   
    l_arc = Arc((5,-25), 47.5, 47.5, theta1=292, theta2=68, lw=lw,
                color=color, zorder=zorder)
    r_arc = Arc((89, -25), 47.5, 47.5, theta1=112, theta2=248, lw=lw,
                color=color, zorder=zorder)

   
    half_court = Rectangle((47,-50), 0, 50, lw=lw, color=color,
                           zorder=zorder)

    hc_big_circle = Circle((47, -25), radius=6, lw=lw, fill=False,
                           color=color, zorder=zorder)
    hc_sm_circle = Circle((47, -25), radius=2, lw=lw, fill=False,
                          color=color, zorder=zorder)

    court_elements = [l_hoop, l_backboard, l_outer_box, outer,
                      l_inner_box, l_free_throw, l_corner_a,
                      l_corner_b, l_arc, r_hoop, r_backboard, 
                      r_outer_box, r_inner_box, r_free_throw,
                      r_corner_a, r_corner_b, r_arc, half_court,
                      hc_big_circle, hc_sm_circle]

   
    for element in court_elements:
        ax.add_patch(element)
    ax.set_facecolor('black')
    return ax

## **Disegno**

### **Disegnare Traiettorie Evento divise per Momenti**

In [ ]:
def marracash(a,events,j,k):
     for i in range(0, a.shape[0], 444):

      dfi=a.iloc[i:i+444]
      pg=dfi[dfi.role=='pg']
      sg=dfi[dfi.role=='sg']
      sf=dfi[dfi.role=='sf']
      pf=dfi[dfi.role=='pf']
      ce=dfi[dfi.role=='c']
      ball=dfi[dfi.role=='ball']

      plt.figure(figsize=(15, 11.5))
      plt.scatter(pg.x_loc, -pg.y_loc, c=pg.game_clock,cmap=plt.cm.Blues_r, s=50, zorder=5)
      plt.scatter(sg.x_loc, -sg.y_loc, c=sg.game_clock, cmap=plt.cm.Oranges_r, s=50, zorder=1)
      plt.scatter(sf.x_loc, -sf.y_loc, c=sf.game_clock,cmap=plt.cm.RdPu_r, s=50, zorder=4)
      plt.scatter(pf.x_loc, -pf.y_loc, c=pf.game_clock,cmap=plt.cm.Purples_r, s=50, zorder=2)
      plt.scatter(ce.x_loc, -ce.y_loc, c=ce.game_clock,cmap=plt.cm.Greens_r, s=50, zorder=3)
      plt.scatter(ball.x_loc, -ball.y_loc, c=ball.game_clock,cmap=plt.cm.Reds_r, s=50, zorder=6)
    

      draw_court()

      plt.xlim(0, 101)
      plt.ylim(-50, 0)
      plt.yticks([])
      plt.xticks([])
      plt.savefig("/content/drive/MyDrive/LabIA/Images/Boston/"+str(k)+'/'+str(events["event_id"][j])+'/'+str(i)+'.png', bbox_inches = 'tight')
      plt.close()
      

### **Operazioni tra dataset**


In [ ]:
def tedua(k,role):
  result=open_file('/content/drive/MyDrive/LabIA/Data/Traiettorie/Boston/1 ('+str(k)+').json',role)
  try:
      os.mkdir("/content/drive/MyDrive/LabIA/Images/Boston/"+str(k))
  except:
      print("Folder already found")
  events=result.groupby("event_id",as_index=False)["moment"].count()
  events=events[events.moment>=3828]
  events=events[["event_id"]]
  result = pd.merge(result,events,on='event_id')
  result.sort_values(by=["moment","player_id"], inplace = True)
  for j in events.index: 
     a=result[result.event_id==events["event_id"][j]]
     c=assign_role(a)
     a = pd.merge(a,c,on='player_id')
     a.drop(['pg%', 'sg%','sf%','pf%','c%'], axis = 1,inplace=True)
     a.sort_values(by=["moment","player_id"], inplace = True)
     try:
      os.mkdir("/content/drive/MyDrive/LabIA/Images/Boston/"+str(k)+'/'+str(events["event_id"][j]))
     except:
      print("Folder already found")
     marracash(a,events,j,k)

### **Scelta Team e Partita**

In [ ]:
headers = ["player_id", "name", "surname", 
           "team", "pg%", "sg%", "sf%","pf%","c%"]
role=pd.read_csv(r'/content/drive/MyDrive/LabIA/Data/Ruoli Giocatori/NBA_Player_Role_2016 Query.csv',sep=';')
role.columns=headers
role=role[role.team=="BOS"]
role=role.append({"player_id":-1,"name":"Ball","surname":None,"team":None,"pg%":-1,"sg%":-1,"sf%":-1,"pf%":-1,"c%":-1}, ignore_index=True)
role.sort_values(by=["player_id"], inplace = True)
tedua(42,role)

### **Generazione Cartelle Dataset divise tra sinistra e destra**

In [ ]:
for u in range(3,43):
  try:
      os.mkdir("/content/drive/MyDrive/LabIA/Images/Boston/3/Indy Rip 14/L/"+str(u))
      os.mkdir("/content/drive/MyDrive/LabIA/Images/Boston/3/Indy Rip 14/R/"+str(u))
      os.mkdir("/content/drive/MyDrive/LabIA/Images/Boston/3/Zip Chase/L/"+str(u))
      os.mkdir("/content/drive/MyDrive/LabIA/Images/Boston/3/Zip Chase/R/"+str(u))
  except:
      print("Folder already found")
  

## **Data Augmentation**

### **NumPy: Flip delle Immagini**

In [ ]:
def image_flip(a,s):
  im = Image.open(a)
  numpydata = np.asarray(im)
  numpydata=np.flip(numpydata,0)
  numpydata=np.flip(numpydata,1)
  #noise = np.random.normal(0, .1, numpydata.shape)
  #numpydata = numpydata + noise
  pilImage = Image.fromarray(numpydata)
  pilImage.save(s)


In [ ]:
i=1
for file in os.listdir('/content/drive/MyDrive/LabIA/Dataset/Temporaneo/CNN/0 Indy Rip 14/R'):
    b='/content/drive/MyDrive/LabIA/Dataset/Temporaneo/CNN/0 Indy Rip 14/R/'+file
    s='/content/drive/MyDrive/LabIA/Dataset/Temporaneo/CNN/0 Indy Rip 14/L/'+str(i)+".png"
    image_flip(b,s)
    i=i+1
    

### **NumPy: Moltiplicazioni delle Immagini**

In [ ]:
def image_hor(a,s,f):
  im = Image.open(a)
  numpydata = np.asarray(im)
  numpydata=np.flip(numpydata,1)
  #noise = np.random.normal(0, .1, numpydata.shape)
  #numpydata = numpydata + noise
  pilImage = Image.fromarray(numpydata)
  im.save(f)
  pilImage.save(s)

def image_ver(a,s,c,d):
  im1 = Image.open(a)
  im2 = Image.open(s)
  numpydata1 = np.asarray(im1)
  numpydata2 = np.asarray(im2)
  numpydata1=np.flip(numpydata1,0)
  numpydata2=np.flip(numpydata2,0)
  #noise = np.random.normal(0, .1, numpydata.shape)
  #numpydata = numpydata + noise
  pilImage1 = Image.fromarray(numpydata1)
  pilImage2 = Image.fromarray(numpydata2)
  pilImage1.save(c)
  pilImage2.save(d)

In [ ]:
i=1999
for file in os.listdir('/content/drive/MyDrive/LabIA/Dataset/Temporaneo/CNN/0 Indy Rip 14/L'):
    b='/content/drive/MyDrive/LabIA/Dataset/Temporaneo/CNN/0 Indy Rip 14/L/'+file
    k='/content/drive/MyDrive/LabIA/Dataset/CNN/Train/Indy Rip/'+file
    s='/content/drive/MyDrive/LabIA/Dataset/CNN/Train/Indy Rip/'+str(i)+".png"
    f='/content/drive/MyDrive/LabIA/Dataset/CNN/Train/Indy Rip/'+str(i+1)+".png"
    g='/content/drive/MyDrive/LabIA/Dataset/CNN/Train/Indy Rip/'+str(i+2)+".png"
    image_hor(b,s,k)
    image_ver(b,s,f,g)
    i=i+3
    